Q. conv1d에서는 GlobalAveragePooling1D()이 더 적합해?  
A.  
네, Conv1D에서는 GlobalAveragePooling1D()이 더 적합합니다.

Conv1D는 시계열 데이터나 1D 데이터에 대한 합성곱 연산을 수행합니다. Conv1D의 출력은 2D 데이터이지만, 시계열 데이터의 경우에는 시간 축에 대한 정보가 중요합니다. 따라서, GlobalAveragePooling1D()을 사용하여 시간 축에 대한 정보를 유지하면서 데이터의 크기를 줄일 수 있습니다.

GlobalAveragePooling1D()은 Conv1D의 출력을 시간 축에 대한 평균을 계산하여 1D 데이터로 변환합니다. 이는 Conv1D의 출력을 Flatten()을 사용하여 1D 데이터로 변환하는 것보다 더 적합합니다. 왜냐하면 Flatten()은 Conv1D의 출력을 단순히 1D 데이터로 변환하지만, GlobalAveragePooling1D()은 시간 축에 대한 정보를 유지하면서 데이터의 크기를 줄이기 때문입니다.

따라서, Conv1D에서는 GlobalAveragePooling1D()이 더 적합합니다.

예를 들어, 다음과 같은 모델을 사용할 수 있습니다.
```python
model = Sequential()
model.add(Conv1D(64, kernel_size=3, activation='relu', input_shape=(n_steps, 8)))
model.add(GlobalAveragePooling1D())
model.add(Dense(128, activation='relu'))
model.add(Dense(5, activation='softmax'))
```
이 모델은 Conv1D의 출력을 GlobalAveragePooling1D()을 사용하여 시간 축에 대한 평균을 계산하고, Dense 층에 입력으로 사용합니다.

# 기본

In [1]:
import pandas as pd
import numpy as np
from collections import Counter
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, GlobalAveragePooling1D, Dense
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

# 데이터 불러오기
df = pd.read_csv('./datasets/abalone.csv', index_col=0)

# 데이터 전처리
df['Sex'] = df['Sex'].map({'M': 0, 'F': 1, 'I': 2})

# 'Rings'를 label로 사용
# 기존에 주어진 label에는 다음과 같은 문제가 있음
# 1) label의 시작이 0이 아니고 1부터임
# 2) label 값 중 28이 비어있음
# 이 문제를 해결하기 위해 label을 새롭게 매핑
rings_mapping = {29: 28}
df['Rings'] = df['Rings'].replace(rings_mapping)

# label을 0 ~ 27로 매핑
rings_mapping = {}
for i in range(len(np.unique(df['Rings']))):
    rings_mapping = {i+1: i}
    df['Rings'] = df['Rings'].replace(rings_mapping)

# label을 범위를 묶어서 총 5개 정도의 label로 다시 구성
label_mapping = {}
for i in range(28):
    if i < 5:
        label_mapping[i] = 0
    elif i < 10:
        label_mapping[i] = 1
    elif i < 15:
        label_mapping[i] = 2
    elif i < 20:
        label_mapping[i] = 3
    else:
        label_mapping[i] = 4
df['Rings'] = df['Rings'].replace(label_mapping)

# dataset을 sequential하게 만들기
def split_sequences_classification_majority(data, labels, n_steps):
    X, y = [], []
    for i in range(len(data) - n_steps + 1):
        seq_x = data[i:i + n_steps]
        seq_labels = labels[i:i + n_steps]
        most_common_label = Counter(seq_labels).most_common(1)[0][0]
        X.append(seq_x)
        y.append(most_common_label)
    return np.array(X), np.array(y)

n_steps = 10
X = df.drop(['Rings'], axis=1)
y = df['Rings']
X, y = split_sequences_classification_majority(X.values, y.values, n_steps)

# 데이터 스케일링
scaler = StandardScaler()
X = scaler.fit_transform(X.reshape(-1, 8)).reshape(-1, n_steps, 8)

# label에 대해 원핫 인코딩
y = to_categorical(y)

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 모델 생성(1) 맥스 풀링 사용
# model = Sequential()
# model.add(Conv1D(64, kernel_size=3, activation='relu', input_shape=(n_steps, 8)))
# model.add(MaxPooling1D(pool_size=2))
# model.add(Flatten())
# model.add(Dense(128, activation='relu'))
# model.add(Dense(5, activation='softmax'))

# 모델 생성(2) 글로벌 풀링 사용
model = Sequential()
model.add(Conv1D(64, kernel_size=3, activation='relu', input_shape=(n_steps, 8)))
model.add(GlobalAveragePooling1D())
model.add(Dense(128, activation='relu'))
model.add(Dense(5, activation='softmax'))

# 모델 컴파일
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 모델 학습
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test))

# 모델 평가
y_pred = model.predict(X_test)
y_pred_class = np.argmax(y_pred, axis=1)
y_test_class = np.argmax(y_test, axis=1)

print('정확도:', accuracy_score(y_test_class, y_pred_class))
print('분류 보고서:')
print(classification_report(y_test_class, y_pred_class))
print('혼동 행렬:')
print(confusion_matrix(y_test_class, y_pred_class))

2024-09-23 23:43:30.398724: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/100


2024-09-23 23:43:34.886117: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


105/105 [==============================] - 1s 5ms/step - loss: 0.8348 - accuracy: 0.6818 - val_loss: 0.6479 - val_accuracy: 0.7374
Epoch 2/100
105/105 [==============================] - 0s 3ms/step - loss: 0.5840 - accuracy: 0.7361 - val_loss: 0.5306 - val_accuracy: 0.7734
Epoch 3/100
105/105 [==============================] - 0s 2ms/step - loss: 0.5239 - accuracy: 0.7567 - val_loss: 0.4992 - val_accuracy: 0.7722
Epoch 4/100
105/105 [==============================] - 0s 1ms/step - loss: 0.5055 - accuracy: 0.7726 - val_loss: 0.4835 - val_accuracy: 0.7854
Epoch 5/100
105/105 [==============================] - 0s 1ms/step - loss: 0.4812 - accuracy: 0.7813 - val_loss: 0.4643 - val_accuracy: 0.7950
Epoch 6/100
105/105 [==============================] - 0s 1ms/step - loss: 0.4666 - accuracy: 0.7879 - val_loss: 0.4518 - val_accuracy: 0.8034
Epoch 7/100
105/105 [==============================] - 0s 1ms/step - loss: 0.4541 - accuracy: 0.7849 - val_loss: 0.4537 - val_accuracy: 0.8070
Epoch 8/100

/opt/anaconda3/envs/gc_dl/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/envs/gc_dl/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/envs/gc_dl/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# 상관계수 높은 feature 삭제 한 경우

In [2]:
import pandas as pd
import numpy as np
from collections import Counter
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, GlobalAveragePooling1D, Dense
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

# 데이터 불러오기
df = pd.read_csv('./datasets/abalone.csv', index_col=0)

# 데이터 전처리
df['Sex'] = df['Sex'].map({'M': 0, 'F': 1, 'I': 2})

# 'Rings'를 label로 사용
# 기존에 주어진 label에는 다음과 같은 문제가 있음
# 1) label의 시작이 0이 아니고 1부터임
# 2) label 값 중 28이 비어있음
# 이 문제를 해결하기 위해 label을 새롭게 매핑
rings_mapping = {29: 28}
df['Rings'] = df['Rings'].replace(rings_mapping)

# label을 0 ~ 27로 매핑
rings_mapping = {}
for i in range(len(np.unique(df['Rings']))):
    rings_mapping = {i+1: i}
    df['Rings'] = df['Rings'].replace(rings_mapping)

# label을 범위를 묶어서 총 5개 정도의 label로 다시 구성
label_mapping = {}
for i in range(28):
    if i < 5:
        label_mapping[i] = 0
    elif i < 10:
        label_mapping[i] = 1
    elif i < 15:
        label_mapping[i] = 2
    elif i < 20:
        label_mapping[i] = 3
    else:
        label_mapping[i] = 4
df['Rings'] = df['Rings'].replace(label_mapping)

# feature 중에서 상관계수가 매우 높은 feature 쌍을 삭제
corr_matrix = df.drop(['Rings'], axis=1).corr()
corr_matrix = corr_matrix.unstack()
corr_matrix = corr_matrix[corr_matrix != 1]
corr_matrix = corr_matrix.sort_values(ascending=False)

high_corr_features = []
for index, value in corr_matrix.items():
    if abs(value) > 0.9:
        high_corr_features.append(index[0])
        high_corr_features.append(index[1])

high_corr_features = list(set(high_corr_features))

df = df.drop(high_corr_features[1:], axis=1)

# dataset을 sequential하게 만들기
def split_sequences_classification_majority(data, labels, n_steps):
    X, y = [], []
    for i in range(len(data) - n_steps + 1):
        seq_x = data[i:i + n_steps]
        seq_labels = labels[i:i + n_steps]
        most_common_label = Counter(seq_labels).most_common(1)[0][0]
        X.append(seq_x)
        y.append(most_common_label)
    return np.array(X), np.array(y)

n_steps = 10
X = df.drop(['Rings'], axis=1)
y = df['Rings']
X, y = split_sequences_classification_majority(X.values, y.values, n_steps)

# 데이터 스케일링
scaler = StandardScaler()
X = scaler.fit_transform(X.reshape(-1, X.shape[2])).reshape(-1, n_steps, X.shape[2])

# label에 대해 원핫 인코딩
y = to_categorical(y)

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 모델 생성
model = Sequential()
model.add(Conv1D(64, kernel_size=3, activation='relu', input_shape=(n_steps, X.shape[2])))
model.add(GlobalAveragePooling1D())
model.add(Dense(128, activation='relu'))
model.add(Dense(5, activation='softmax'))

# 모델 컴파일
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 모델 학습
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test))

# 모델 평가
y_pred = model.predict(X_test)
y_pred_class = np.argmax(y_pred, axis=1)
y_test_class = np.argmax(y_test, axis=1)

print('정확도:', accuracy_score(y_test_class, y_pred_class))
print('분류 보고서:')
print(classification_report(y_test_class, y_pred_class))
print('혼동 행렬:')
print(confusion_matrix(y_test_class, y_pred_class))

Epoch 1/100
105/105 [==============================] - 1s 2ms/step - loss: 0.8748 - accuracy: 0.6911 - val_loss: 0.7013 - val_accuracy: 0.7074
Epoch 2/100
105/105 [==============================] - 0s 1ms/step - loss: 0.6940 - accuracy: 0.7037 - val_loss: 0.6442 - val_accuracy: 0.6847
Epoch 3/100
105/105 [==============================] - 0s 1ms/step - loss: 0.6438 - accuracy: 0.7127 - val_loss: 0.6175 - val_accuracy: 0.6978
Epoch 4/100
105/105 [==============================] - 0s 1ms/step - loss: 0.6171 - accuracy: 0.7160 - val_loss: 0.6078 - val_accuracy: 0.6906
Epoch 5/100
105/105 [==============================] - 0s 1ms/step - loss: 0.5976 - accuracy: 0.7184 - val_loss: 0.5980 - val_accuracy: 0.7230
Epoch 6/100
105/105 [==============================] - 0s 1ms/step - loss: 0.5774 - accuracy: 0.7418 - val_loss: 0.5828 - val_accuracy: 0.7074
Epoch 7/100
105/105 [==============================] - 0s 1ms/step - loss: 0.5662 - accuracy: 0.7379 - val_loss: 0.5719 - val_accuracy: 0.7194

/opt/anaconda3/envs/gc_dl/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/envs/gc_dl/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/envs/gc_dl/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
